In [2]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Dropout, Dense, Flatten
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.metrics import Precision, Recall
from skimage.color import rgb2lab, lab2rgb
from skimage import exposure
import numpy as np
import os
from cv2 import imread, imshow, destroyAllWindows, waitKey, resize

In [1]:

processed_images_X_List = []
processed_images_Y_List = []
processed_images_X_List1 = []
processed_images_Y_List1 = []
processed_images_X_List2 = []
processed_images_Y_List2 = []
processed_images_X_List3 = []
processed_images_Y_List3 = []
processed_images_X_List4 = []
processed_images_Y_List4 = []

In [3]:

def load_data(data_dir, size):
    count = 0
    for filename in os.listdir(data_dir):
        image = imread(os.path.join(data_dir, filename))
        image = resize(image, dsize=size)
        image = np.array(image, dtype=float)
        X = rgb2lab(1.0/255*image)[:,:,0]
        Y = rgb2lab(1.0/255*image)[:,:,1:]
        Y /= 128
        processed_images_X_List.append(X)
        processed_images_Y_List.append(Y)
        count += 1
        print(count, " - ", filename)
        if count == 4000:
            print("Durduruldu")
            break
    
    print("Dosyalar Yüklendi")

load_data("F:\\Restoresyon\\data\\humans", (400,400))


1  -  Aaron_Eckhart_0001.jpg
2  -  Aaron_Guiel_0001.jpg
3  -  Aaron_Patterson_0001.jpg
4  -  Aaron_Peirsol_0001.jpg
5  -  Aaron_Peirsol_0002.jpg
6  -  Aaron_Peirsol_0003.jpg
7  -  Aaron_Peirsol_0004.jpg
8  -  Aaron_Pena_0001.jpg
9  -  Aaron_Sorkin_0001.jpg
10  -  Aaron_Sorkin_0002.jpg
11  -  Aaron_Tippin_0001.jpg
12  -  Abbas_Kiarostami_0001.jpg
13  -  Abba_Eban_0001.jpg
14  -  Abdel_Aziz_Al-Hakim_0001.jpg
15  -  Abdel_Madi_Shabneh_0001.jpg
16  -  Abdel_Nasser_Assidi_0001.jpg
17  -  Abdel_Nasser_Assidi_0002.jpg
18  -  Abdoulaye_Wade_0001.jpg
19  -  Abdoulaye_Wade_0002.jpg
20  -  Abdoulaye_Wade_0003.jpg
21  -  Abdoulaye_Wade_0004.jpg
22  -  Abdulaziz_Kamilov_0001.jpg
23  -  Abdullah_0001.jpg
24  -  Abdullah_0002.jpg
25  -  Abdullah_0003.jpg
26  -  Abdullah_0004.jpg
27  -  Abdullah_Ahmad_Badawi_0001.jpg
28  -  Abdullah_al-Attiyah_0001.jpg
29  -  Abdullah_al-Attiyah_0002.jpg
30  -  Abdullah_al-Attiyah_0003.jpg
31  -  Abdullah_Gul_0001.jpg
32  -  Abdullah_Gul_0002.jpg
33  -  Abdullah_Gul_0

In [6]:
processed_images_X_List1 = processed_images_X_List[:1000]
processed_images_Y_List1 = processed_images_Y_List[:1000]
processed_images_X_List2 = processed_images_X_List[1001:2000]
processed_images_Y_List2 = processed_images_Y_List[1001:2000]
processed_images_X_List3 = processed_images_X_List[2001:3000]
processed_images_Y_List3 = processed_images_Y_List[2001:3000]
processed_images_X_List4 = processed_images_X_List[3001:4000]
processed_images_Y_List4 = processed_images_X_List[3001:4000]

In [7]:
len(processed_images_X_List1)

1000

In [8]:
processed_images_X = np.array(processed_images_X_List1)
processed_images_Y = np.array(processed_images_Y_List1)

MemoryError: Unable to allocate 1.19 GiB for an array with shape (1000, 400, 400) and data type float64

In [ ]:
processed_images_X = processed_images_X.reshape(-1, 400, 400, 1)
processed_images_Y = processed_images_Y.reshape(-1, 400, 400, 2)

In [ ]:
model = Sequential()
model.add(InputLayer(shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

model.compile(optimizer='rmsprop',loss='mse')

callbacks = [EarlyStopping(monitor="val_loss", patience=50)]



In [ ]:
model.fit(x=processed_images_X, 
    y=processed_images_Y,
    callbacks=callbacks,
    batch_size=1,
    epochs=100)

In [ ]:
def color_me(image1):
    image1 = resize(image1, (400,400))
    image1 = np.array(image1, dtype=float)
    X = rgb2lab(1.0/255*image1)[:,:,0]
    
    X = X.reshape(1, 400, 400, 1)
    

    output = model.predict(X)
    output = output*128
    cur = np.zeros((400, 400, 3))
    cur[:,:,0] = X[0][:,:,0]
    cur[:,:,1:] = output[0]

    return cur
def color_me2(image1):
    original_shape = image1.shape[:2]
    image1 = np.array(image1, dtype=float)
    X = rgb2lab(1.0/255*image1)[:, :, 0]

    X = X.reshape(1, original_shape[0], original_shape[1], 1)

    output = model.predict(X)
    output = output * 128

    # Eğer model çıktısı doğrudan uygun şekilde değilse, burada bir yeniden boyutlandırma yapılabilir.
    # Örneğin:
    # output_resized = resize(output[0], (original_shape[0], original_shape[1], 2))

    cur = np.zeros((original_shape[0], original_shape[1], 3))
    cur[:, :, 0] = X[0, :, :, 0]
    # Burada output yerine output_resized kullanılabilir.
    cur[:, :, 1:] = output[0][:original_shape[0], :original_shape[1], :]

    return cur


In [ ]:
def equalize_histogram(image):
    
    return exposure.equalize_hist(image)

In [ ]:


get_image = imread("Test/test7.jpeg")
#get_image = resize(get_image, (get_image.shape[1]//3, get_image.shape[0]//3))
cur = color_me2(get_image)
image = lab2rgb(cur)

#imagee = rgb2gray(lab2rgb(cur))
imshow("Color me", image)
imshow("original", get_image)
waitKey()
destroyAllWindows()

In [ ]:
model.summary()

In [ ]:
model.save("humans.h5")